In [8]:
# Packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import torchvision
import imp
import matplotlib as mpl
from matplotlib import cm
import statsmodels.tsa.stattools as stat
from sklearn.model_selection import KFold
import time

# Set Latex font

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [9]:
# Preprocessing
def preprocessing_covertype(x_train,x_test,y_train,y_test,c1,c2):

    # Making sure that the values are float so that we can get decimal points 
    # after division
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    y_train = y_train.astype('int')
    y_test = y_test.astype('int')

    # Select the classes to classify for binary logistic regression
    i1 = np.where((y_train == c1) | (y_train == c2))
    y1_train = y_train[i1]
    x1_train = x_train[i1]
    i1 = np.where((y_test == c1) | (y_test == c2))
    y1_test = y_test[i1]
    x1_test = x_test[i1]

    # Replace labels of the current class with 1 and other labels with 0
    i1 = np.where(y1_train == c1)
    i2 = np.where(y1_train == c2)
    y1_train[i1] = 1
    y1_train[i2] = 0
    i1 = np.where(y1_test == c1)
    i2 = np.where(y1_test == c2)
    y1_test[i1] = 1
    y1_test[i2] = 0
    
    # Return objects of interest
    return x1_train, x1_test, y1_train, y1_test;

MNIST dataset

Covertype dataset

In [10]:
# Vector of classes to compare
c1 = 5
c2 = 3

# Import data
from sklearn import datasets, preprocessing
dataset = datasets.fetch_covtype()
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(dataset.data)
x_train = X[0:500000,:]
x_test = X[500000:,:]
y_train = dataset.target[0:500000]
y_test = dataset.target[500000:]

# Data processing
X, X_test, y, y_test = preprocessing_covertype(x_train,x_test,y_train,y_test,c1,c2)

# Initialization
d = np.size(X,1)
n = np.size(y)
tau = 1 # regularization parameter

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

#### MALA

In [4]:
# Parameters
w, v = np.linalg.eig(np.dot(X.T,X))
w = np.max(w.real)
M = w * (1/4) + tau 
gamma = 0.01/M
T = 300000

# Initialization
theta_MALA = np.zeros((T,d))
start_time = time.time()
U_MALA = np.zeros(T)

for t in range(T-1):
    P = 1 / (1 + np.exp(-np.dot(X,theta_MALA[t,:])))
    grad = np.dot(X.T, np.reshape(P - y,(n,))) + tau * theta_MALA[t,:]
    grad = np.reshape(grad,(d,))
    s = np.zeros(d)
    s = theta_MALA[t,:] - gamma * grad + np.sqrt(2*gamma)*np.random.normal(0,1,d)
    
    u = np.random.uniform()
    P = 1 / (1 + np.exp(-np.dot(X,s)))
    U = - np.dot(y,np.log(P)) - np.dot(1-y,np.log(1-P)) + tau/2 * np.linalg.norm(s)**2
    P = 1 / (1 + np.exp(-np.dot(X,theta_MALA[t,:])))
    U_old = - np.dot(y,np.log(P)) - np.dot(1-y,np.log(1-P)) + tau/2 * np.linalg.norm(theta_MALA[t,:])**2
    P = 1 / (1 + np.exp(-np.dot(X,s)))
    grad_prop = np.dot(X.T, np.reshape(P - y,(n,1))) + tau * s
    grad_prop = np.reshape(grad,(d,))
    ratio = np.exp(-U + U_old \
                   - (1/(4*gamma))*np.linalg.norm(theta_MALA[t,:] - s + gamma * grad_prop,2)**2 \
                   + (1/(4*gamma))*np.linalg.norm(s-theta_MALA[t,:] + gamma * grad,2)**2)
    print(min(1,ratio))
    if u <= ratio:
        theta_MALA[t+1,:] = s
    else:
        theta_MALA[t+1,:] = theta_MALA[t,:] 
        
    # Compute the neg. log posterior
    P = 1 / (1 + np.exp(-np.dot(X,theta_MALA[t+1,:])))
    P = P - 1e-5
    P[P<0] = 1e-10
    U_MALA[t] = - np.dot(y,np.log(P)) - np.dot(1-y,np.log(1-P)) + tau/2 * np.linalg.norm(theta_MALA[t+1,:])**2
                
    if t % 1000 == 0:
        print(t)
             
deltaT = time.time() - start_time
print("--- %s seconds ---" % deltaT)    

NameError: name 'X' is not defined

#### DG-LMC

In [23]:
# Parameters
Mi = 1/4
rho = 1/(50*Mi)
gamma = 0.05*rho/(rho*Mi+1)
T = 200000
N = 1
tau = 1

# Pre-processing
C = np.linalg.inv(np.dot(X.T,X)/rho + tau*np.eye(d))

# Initialization
z = np.zeros(n)
theta = np.zeros((T,d))
U = np.zeros(T)

for t in range(T-1):
    
    # N ULA steps on each worker
    for k in range(N):
        z = (1 - gamma/rho) * z + (gamma/rho) * X.dot(theta[t,:]) - gamma * (1 / (1 + np.exp(-z)) - y) \
          + np.sqrt(2*gamma)*np.random.normal(0,1,size=n)
    
    # Sample theta
    eta1 = z + np.sqrt(rho) * np.random.normal(0,1,size=n)
    eta2 = (1/np.sqrt(tau)) * np.random.normal(0,1,size=d)
    eta = np.dot(X.T,eta1)/rho + eta2*tau
    theta[t+1,:] = np.dot(C,eta)
    
    # Compute the neg. log posterior
    P = 1 / (1 + np.exp(-np.dot(X,theta[t+1,:])))
    P = P - 1e-5
    P[P<0] = 1e-10
    U[t] = - np.dot(y,np.log(P)) - np.dot(1-y,np.log(1-P)) + tau/2 * np.linalg.norm(theta[t+1,:])**2
    
    if t % 10000 == 0:
        print(t)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000


#### D-SGLD

In [107]:
# Parameters
b = 16
X_split = np.array_split(X, b)
y_split = np.array_split(y, b)
w, v = np.linalg.eig(np.dot(X.T,X))
w = np.max(w.real)
M = w * (1/4) + tau 
gamma = 1/(50*M)
T = 200000
N = 10
n_shard = int(n/b)
batch_size = int(n_shard/10)

# Initialization
theta_SGLD = np.zeros((T,b,d))
z = np.zeros((b,d))
U_SGLD = np.zeros(T)

for t in range(T-1):
    
    # Assign a random shard to each worker
    ind = np.random.permutation(b)

    for i in range(b):
        
        # Sample a mini-batch on shard ind[i]
        idx_subset = np.random.choice(n_shard, size=batch_size, replace=False)
        P = 1 / (1 + np.exp(-np.dot(X_split[ind[i]][idx_subset],z[i,:])))
        grad = np.dot(X_split[ind[i]][idx_subset].T, np.reshape(P - y_split[ind[i]][idx_subset],(batch_size,1)))
        grad = np.reshape(grad,(d,))
            
        # Perform N local iterations
        for k in range(N):
            z[i,:] = z[i,:] - gamma * (n_shard*b/batch_size * grad + tau*z[i,:]) \
                   + np.sqrt(2*gamma)*np.random.normal(0,1,size=d)
            
    # Gather the end of the trajectory
    theta_SGLD[t+1,:,:] = z
    
    # Compute the neg. log posterior
    P = 1 / (1 + np.exp(-np.dot(X,theta_SGLD[t+1,0,:])))
    P = P - 1e-5
    P[P<0] = 1e-10
    U_SGLD[t] = - np.dot(y,np.log(P)) - np.dot(1-y,np.log(1-P)) + tau/2 * np.linalg.norm(theta_SGLD[t+1,0,:])**2

    if t % 10000 == 0:
        print(t)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000


Global consensus Monte Carlo

In [112]:
# Parameters
Mi = 1/4
rho = 1/(5*Mi)
gamma = 0.0001*rho/(rho*Mi+1)
T = 200000
N = 1
tau = 1

# Pre-processing
C = np.linalg.inv(np.dot(X.T,X)/rho + tau*np.eye(d))

# Initialization
z = np.zeros(n)
theta_GCMC = np.zeros((T,d))
U_GCMC = np.zeros(T)

for t in range(T-1):
    
    # N MALA steps on each worker
    for k in range(N):
        grad = (1/rho) * (z- X.dot(theta_GCMC[t,:])) + (1 / (1 + np.exp(-z)) - y)
        s = z - gamma * grad + np.sqrt(2*gamma)*np.random.normal(0,1,size=n)
        
        u = np.random.uniform()
        P = 1 / (1 + np.exp(-s))
        U = - np.dot(y,np.log(P)) - np.dot(1-y,np.log(1-P)) + 1/(2*rho) * np.linalg.norm(s - X.dot(theta_GCMC[t,:]))**2
        P = 1 / (1 + np.exp(-z))
        U_old = - np.dot(y,np.log(P)) - np.dot(1-y,np.log(1-P)) + 1/(2*rho) * np.linalg.norm(z - X.dot(theta_GCMC[t,:]))**2
        P = 1 / (1 + np.exp(-s))
        grad_prop = P - y + 1/rho * (s - X.dot(theta_GCMC[t,:]))
        grad_prop = np.reshape(grad,(n,))
        ratio = np.exp(-U + U_old \
                   - (1/(4*gamma))*np.linalg.norm(z - s + gamma * grad_prop,2)**2 \
                   + (1/(4*gamma))*np.linalg.norm(s - z + gamma * grad,2)**2)
        if t % 1 == 0:
            print(ratio)
        if u <= ratio:
            z = s
           
    # Sample theta
    eta1 = z + np.sqrt(rho) * np.random.normal(0,1,size=n)
    eta2 = (1/np.sqrt(tau)) * np.random.normal(0,1,size=d)
    eta = np.dot(X.T,eta1)/rho + eta2*tau
    theta_GCMC[t+1,:] = np.dot(C,eta)
    
    # Compute the neg. log posterior
    P = 1 / (1 + np.exp(-np.dot(X,theta[t+1,:])))
    P = P - 1e-5
    P[P<0] = 1e-10
    U_GCMC[t] = - np.dot(y,np.log(P)) - np.dot(1-y,np.log(1-P)) + tau/2 * np.linalg.norm(theta_GCMC[t+1,:])**2
    
    if t % 10000 == 0:
        print(t)

0.02457246350788605
0
0.025732828948452927
0.023740281229265973
0.023971670585973912
0.022538642565304266
0.02408517570740166
0.023822740888462927
0.02371622931967897
0.02379810728072194
0.02300801124716607
0.024403540841731144
0.025142948099961943
0.024798555770912955
0.02360837494258711
0.02512311899723084
0.02358043019673124
0.023570972845307514
0.022621055196846478
0.023215442137320134
0.02191738998038083
0.02529203458001063
0.024785249108090778
0.02492473144825535
0.024115154671926774
0.024628076241776063
0.024403696929577574
0.024792782576438906
0.023532096344118562
0.023464846527081584
0.023407464455604574
0.023991575358528427
0.024342761715733227
0.024957146854333003
0.024202252424605233
0.023628918278622894
0.023896052668015828
0.024803730984895384
0.023472626540970404
0.02398259249337549
0.023718873429521902
0.023769799951685648
0.02390857921621052
0.02366891710021454
0.02329898072160576
0.023603375666642935
0.024902895070990503
0.02371574752460124
0.02232396051305525
0.02493

KeyboardInterrupt: 